In [ ]:
%gui tk
import tkinter as tk
from tkinter import ttk, messagebox
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import heapq  
import pandas as pd


df = pd.read_excel("tramvaji.xlsx")


G = nx.Graph()
tram_colors = {}
colors = ["red", "blue", "green", "orange", "purple","yellow","pink","cyan"]

for index, row in df.iterrows():
    G.add_edge(row["Pocetak"], row["Kraj"], weight=row["Vrijeme"], tram=row["Tramvaj"])
    if row["Tramvaj"] not in tram_colors:
        tram_colors[row["Tramvaj"]] = colors[len(tram_colors) % len(colors)]


def heuristic(a, b):
    
    if G.has_edge(a, b):
        return G[a][b]['weight']
    else:
        return float('inf')

def greedy_heuristic(a, b):
    return 0

def astar_path(G, start, goal, weight="weight"):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float("inf") for node in G.nodes}
    g_score[start] = 0
    f_score = {node: float("inf") for node in G.nodes}
    f_score[start] = heuristic(start, goal)

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for neighbor in G.neighbors(current):
            cekanje_g_score = g_score[current] + G[current][neighbor].get(weight, 1) + 1  # 1 min cekanja
            if cekanje_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = cekanje_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []


def greedy_path(G, start, goal, weight="weight"):
    open_set = [start]
    came_from = {}
    g_score = {node: float("inf") for node in G.nodes}
    g_score[start] = 0

    while open_set:
        current = min(open_set, key=lambda node: g_score[node] + greedy_heuristic(node, goal))
        open_set.remove(current)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for neighbor in G.neighbors(current):
            cekanje_g_score = g_score[current] + G[current][neighbor].get(weight, 1) + 1  # 1 min cekanja
            if cekanje_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = cekanje_g_score
                open_set.append(neighbor)

    return []



def prikazi_graf():
    global canvas, fig, ax, pos
    fig, ax = plt.subplots(figsize=(6, 6))
    pos = nx.spring_layout(G, k=1.4)
    for tram, color in tram_colors.items():
        edges = [(u, v) for u, v, d in G.edges(data=True) if d['tram'] == tram]
        nx.draw_networkx_edges(G, pos, edgelist=edges, width=2, edge_color=color, ax=ax)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=8, ax=ax)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
    canvas = FigureCanvasTkAgg(fig, master=frame_graf)
    canvas.draw()
    canvas.get_tk_widget().pack()
    canvas.get_tk_widget().bind("<MouseWheel>", on_mouse_scroll)
    canvas.get_tk_widget().bind("<ButtonPress-1>", on_mouse_press)
    canvas.get_tk_widget().bind("<ButtonRelease-1>", on_mouse_release)
    canvas.get_tk_widget().bind("<B1-Motion>", on_mouse_move)


def nadji_rutu():
    start = dropdown_start.get()
    cilj = dropdown_cilj.get()
    algoritam = dropdown_algoritam.get()

    if start == cilj:
        messagebox.showerror("Greska", "Pocetna i krajnja stanica su iste!")
        return

    try:
        status_text.delete(1.0, tk.END)
        progress["mode"] = "indeterminate"
        progress.start(10)
        update_status(f"Pocetak pretrage od {start} do {cilj} koristeci {algoritam} algoritam...")

        if algoritam == "Dijkstra":
            ruta = nx.dijkstra_path(G, start, cilj, weight='weight')
        elif algoritam == "A*":
            ruta = astar_path(G, start, cilj, weight='weight')
        elif algoritam == "Greedy":
            ruta = greedy_path(G, start, cilj, weight='weight')
        else:
            messagebox.showerror("Greska", "Nepoznat algoritam.")
            return

        progress["mode"] = "determinate"
        ukupno_vrijeme = sum(G[ruta[i]][ruta[i+1]]['weight'] + 1 for i in range(len(ruta) - 1))

        tramvaji= set()
        for i in range(len(ruta) - 1):
            tramvaji.add(G[ruta[i]][ruta[i+1]]['tram'])

        tramvaji_broj = ', '.join(map(str, tramvaji))

        for i in range(len(ruta)):
            time.sleep(0.5)
            progress["value"] += 100 / len(ruta)
            update_status(f"Obradjuje cvor: {ruta[i]}")

        messagebox.showinfo("Najkraca ruta", f"Optimalna ruta: {' -> '.join(ruta)}\n"
                                             f"Ukupno vrijeme: {ukupno_vrijeme} min\n"
                                             f"Tramvajske linije: {tramvaji_broj}")
    except nx.NetworkXNoPath:
        messagebox.showerror("Greska", "Nema dostupne rute izmedju stanica.")

def update_status(message):
    status_text.insert(tk.END, message + "\n")
    status_text.see(tk.END)
    root.update_idletasks()

def on_mouse_scroll(event):
    scale_factor = 1.2 if event.delta < 0 else 0.8
    ax.set_xlim(ax.get_xlim()[0] * scale_factor, ax.get_xlim()[1] * scale_factor)
    ax.set_ylim(ax.get_ylim()[0] * scale_factor, ax.get_ylim()[1] * scale_factor)
    canvas.draw()

def on_mouse_press(event):
    global press_pos
    press_pos = (event.x, event.y)

def on_mouse_release(event):
    global press_pos
    press_pos = None

def on_mouse_move(event):
    global press_pos
    if press_pos is not None:
        dx = event.x - press_pos[0]
        dy = event.y - press_pos[1]
        ax.set_xlim(ax.get_xlim()[0] - dx / 100, ax.get_xlim()[1] - dx / 100)
        ax.set_ylim(ax.get_ylim()[0] + dy / 100, ax.get_ylim()[1] + dy / 100)
        press_pos = (event.x, event.y)
        canvas.draw()


def exit_app():
    root.destroy()


root = tk.Tk()
root.title("Planer Tramvajskih Ruta")

frame_graf = tk.Frame(root)
frame_graf.pack(side=tk.LEFT, padx=20, pady=20)
prikazi_graf()

frame_options = tk.Frame(root)
frame_options.pack(side=tk.RIGHT, padx=20, pady=20)

tk.Label(frame_options, text="Pocetna stanica:").pack()
dropdown_start = ttk.Combobox(frame_options, values=list(G.nodes))
dropdown_start.pack()

tk.Label(frame_options, text="Krajnja stanica:").pack()
dropdown_cilj = ttk.Combobox(frame_options, values=list(G.nodes))
dropdown_cilj.pack()

tk.Label(frame_options, text="Odaberite algoritam:").pack()
dropdown_algoritam = ttk.Combobox(frame_options, values=["Dijkstra", "A*","Greedy"])
dropdown_algoritam.pack()

btn_nadji_rutu = tk.Button(frame_options, text="Pronadi rutu", command=nadji_rutu)
btn_nadji_rutu.pack(pady=10)

progress = ttk.Progressbar(frame_options, orient="horizontal", length=200, mode="determinate")
progress.pack(pady=10)

tk.Label(frame_options, text="Status pretrage:").pack()
status_text = tk.Text(frame_options, height=10, width=40)
status_text.pack()

btn_exit = tk.Button(frame_options, text="Izlaz", command=exit_app)
btn_exit.pack(pady=10)

root.mainloop()